<a href="https://colab.research.google.com/github/zahid47/sasayaki/blob/main/sasayaki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U -q openai-whisper gradio easynmt edge-tts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB

In [ ]:
import whisper
import requests
import asyncio
import edge_tts
import gradio as gr
from easynmt import EasyNMT

In [ ]:
transcription_model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 116MiB/s]


In [ ]:
translation_model = EasyNMT('mbart50_m2m')

24.9kB [00:00, 27.9MB/s]                   


In [ ]:
VOICE_OUTPUT_FILE = "result.mp3"

In [ ]:
def get_audio(voice):
    audio = whisper.load_audio(voice)

    # trim audio to fit 30 seconds
    audio = whisper.pad_or_trim(audio)

    return audio

In [ ]:
def get_language(model, mel, og_lang="auto"):
    if og_lang != "auto":
        return og_lang

    # detect the spoken language
    _, probs = model.detect_language(mel)
    detected_lang = max(probs, key=probs.get)

    supported_langs = ["en", "ja"]

    if detected_lang in supported_langs:
      return detected_lang
    return "en"

In [ ]:
def translate(text, language):
    return translation_model.translate(text, target_lang=language)

In [ ]:
async def tts(text, language):
    voice = "ja-JP-KeitaNeural" if language == "ja" else "en-AU-WilliamNeural"
    communicate = edge_tts.Communicate(text, voice)
    await communicate.save(VOICE_OUTPUT_FILE)

In [ ]:
def transcribe(audio, selected_language):
  # make log-Mel spectrogram and move to the same device as the model
  mel = whisper.log_mel_spectrogram(audio).to(transcription_model.device)

  language = get_language(transcription_model, mel, selected_language)

  # decode the audio
  options = whisper.DecodingOptions(language=language)
  result = whisper.decode(transcription_model, mel, options)

  return result.text, language

In [ ]:
async def main(voice, selected_language):
    audio = get_audio(voice)

    transcribed, language = transcribe(audio, selected_language)

    target_lang = "ja" if language == "en" else "en"

    translated = translate(transcribed, target_lang)

    await tts(translated, target_lang)

    return language, transcribed, translated, VOICE_OUTPUT_FILE

In [ ]:
app = gr.Interface(
    title='sasayaki',
    allow_flagging="never",
    fn=main,
    inputs=[
        gr.Audio(source="microphone", type="filepath", label="Voice"),
        gr.Radio([("Auto detect", "auto"), ("English", "en"),
                 ("Japanese", "ja")], value="auto", label="Voice Language"),
    ],
    outputs=[
        gr.Textbox(label="Original Language"),
        gr.Textbox(label="Transcript"),
        gr.Textbox(label="Translated transcript"),
        gr.Audio(label="Translated voice")
    ],
)
app.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://50a814131f93841788.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://50a814131f93841788.gradio.live
